# 7 高等的整列

## 整列の時間計算量


<img src="https://ndlsearch.ndl.go.jp/thumbnail/9784062773010.jpg" width="100">

> 「この8枚の金貨の中に、1枚たけ、偽物が混じっています」<br>パスカルは楽しそうに水の充満したペットボトルを押しながら出題を始めた。<br>「偽物は他の7枚より、ほんの少しだけ軽いのです。さて、確実に1マイの偽物を見つけるのに、最低この天秤を何回使えばいいでしょう？」<br>（青柳碧人『浜村渚の計算ノート 3と1/2さつめ』（講談社, 2012）

閑話休題

二つの数の比較による整列の時間計算量を情報理論的に考察する。

## 準備


Pythonで，リストの一部をリストにする方法

In [ ]:
A = [3, 1, 4, 1, 5, 9]
left  = A[:3] # 新しいリストができる。
right = A[3:] # 新しいリストができる。

print(left)
print(right)

left[0] = 1000 # leftを更新しても
print(A)       # Aは元のまま

## 7.1 マージソート


-   アルゴリズム図鑑（有料）
-   [https://ja.wikipedia.org/wiki/マージソート](https://ja.wikipedia.org/wiki/%E3%83%9E%E3%83%BC%E3%82%B8%E3%82%BD%E3%83%BC%E3%83%88)

問題：[ALDS1_5_B](https://onlinejudge.u-aizu.ac.jp/courses/lesson/1/ALDS1/5/ALDS1_5_B) (Merge Sort)

In [ ]:
%%writefile input.dat
10
8 5 9 2 6 3 7 1 10 4

In [ ]:
%%writefile test.py
def merge(A, left, mid, right):
  L = A[left:mid] + [10**10]
  R = A[mid:right] + [10**10]
  i = 0
  j = 0
  for k in range(left, right):
    global cnt  # global変数で数える
    cnt += 1
    if L[i] <= R[j]: #左が小さかったら
      A[k] = L[i]    #左を採用して
      i += 1         #左は次に進む
    else:            #右が小さかったら
      A[k] = R[j]    #右を採用して
      j += 1         #右は次に進む

def mergeSort(A, left, right):
  if left + 1 < right:            #未処理がのっこている
    mid = (left + right) // 2    #真ん中を基準に（//は整数の中で）
    mergeSort(A, left, mid)      #左側をマージソート
    mergeSort(A, mid, right)     #右側をマージソート
    merge(A, left, mid, right)   #マージ

n = int(input())
A = list(map(int, input().split()))
cnt = 0 # 比較回数を数える変数
mergeSort(A, 0, n)
print(*A)
print(cnt)

In [ ]:
!python3 test.py < input.dat

## 準備


7.2節と7.3節でクイックソートが完成する。それを試す前に，クイックソートと同じ分割統治法による整列を扱う。

-   アルゴリズム図鑑（有料）
-   [https://ja.wikipedia.org/wiki/クイックソート](https://ja.wikipedia.org/wiki/%E3%82%AF%E3%82%A4%E3%83%83%E3%82%AF%E3%82%BD%E3%83%BC%E3%83%88)

リストを，最後の要素を基準にして，left, middle, rightに分割する。

In [ ]:
A = [13, 19, 9, 5, 12, 8, 7, 4, 21, 2, 6, 11]

v = A[-1] # 最後の要素
left   = [x for x in A if x <  v]
middle = [x for x in A if x == v]
right  = [x for x in A if x >  v]
print(left, middle, right)

関数にする。

In [ ]:
def partition(A):
  v = A[-1]
  left   = [x for x in A if x <  v]
  middle = [x for x in A if x == v]
  right  = [x for x in A if x >  v]
  return left, middle, right

partition([13, 19, 9, 5, 12, 8, 7, 4, 21, 2, 6, 11])

再帰を使って整列する。

In [ ]:
def sort(A):
  if len(A) < 2: return A
  left, middle, right = partition(A)
  return sort(left) + middle + sort(right)

sort([13, 19, 9, 5, 12, 8, 7, 4, 21, 2, 6, 11])

次のようにまとめれば，整列は4行で書けることになる。

In [ ]:
def sort(A):
  if len(A) < 2: return A
  v = A[-1]
  return sort([x for x in A if x <  v]) + [x for x in A if x == v] + sort([x for x in A if x >  v])

sort([13, 19, 9, 5, 12, 8, 7, 4, 21, 2, 6, 11])

**注意：これは標準的なクイックソートではない。** 通常，クイックソートは次のようなソートだと思われている。♠[C.A.R. Hoareの原論文](https://doi.org/10.1093/comjnl/5.1.10)のものだけがクイックソートだとは思われていない。教科書ではLomutoによる分割を採用している。\[コルメン（CLRS）\]も同様。

> クイックソートの特徴は，（スタック領域を少ししか使わないので）その場で整列できること，$N$ 個の項目の整列に平均約 $N\log N$ 回の操作しか必要でないこと，内側のループが極端に短く書けることである。一方，欠点としては，安定でなく，最悪の場合約 $N^2$ に比例する回数の操作が必要である。（\[セジウィック\] p.273）

整理すると次のとおり。

1.  （長所：高速）平均の時間計算量が $O(n \log n)$ の高速な整列アルゴリズムである。♠定数部分を考えると，マージソートよりも高速である。
2.  （長所：省メモリ）追加のメモリをほとんど使わない。空間計算量は $O(\log n)$ で，マージソートの $O(n)$ よりも少ない。
3.  （短所：破壊的）元のデータを破壊する。この性質をインプレース（in place）ということがある。
4.  （短所：不安定）安定ではない。つまり，キーが等しい要素の順序が保存されない。
5.  （短所）最悪の時間計算量は $O(n^2)$ である。ただしこの短所は，分割の基準を工夫することで解決できる。

先の実装は，（♠を除く）1，5は当てはまるが，2，3，4は当てはまらない。つまり，高速だが，省メモリでなく，破壊的でなく，安定である。標準的なクイックソートに対して，メモリを多く使う代わりに，安定性を得たと言える。（メモリを多く使ってよいなら，マージソートにすれば，安定性に加えて非破壊性も得られる。）

♠追加のメモリが $O(1)$ であることをインプレースということもある。追加のメモリが $O(\log n)$ であるクイックソートは，その意味ではインプレースではない。いずれにしても，破壊的な操作が不可能な純粋関数型言語ではインプレースにはできない。[Haskellの有名な“クイックソート”](https://wiki.haskell.org/Introduction#Quicksort_in_Haskell)は，$O(n)$ のメモリを必要とする。

入門的な教科書では次のような実装もある。いずれも標準的なクイックソートではない。

-   3個の内包の代わりに，1個のfor文で3個のリストにappendする。先の実装より高速だが，省メモリでなく，破壊的でなく，安定である。
-   `left = [x for x in A if x <= v]`とする。先の実装より高速だが，省メモリでなく，破壊的でなく，不安定である。

一般的でない実装で学ぶと，次のような問題で混乱するかもしれない。

> 8.3-2 次のソーティングアルゴリズムのうち安定なものはどれか？挿入ソート，マージソート，ヒープソート，クイックソート。任意の比較ソートアルゴリズムを安定なものに修正するための簡単な方法を与えよ。この方法にはどの程度の余分な時間と領域が必要か？　\[コルメン（CLRS）\]

## 7.2 パーティション


問題：[ALDS1_6_B](https://onlinejudge.u-aizu.ac.jp/courses/lesson/1/ALDS1/6/ALDS1_6_B) (Partition)

## 7.3 クイックソート


問題：[ALDS1_6_C](https://onlinejudge.u-aizu.ac.jp/courses/lesson/1/ALDS1/6/ALDS1_6_C) (Quick Sort)

### 準備


データを読み込んで，安定なソートで整列してみる（Pythonの`sorted`は安定）。

In [ ]:
%%writefile input.dat
6
D 3
H 2
D 1
S 3
D 2
C 1

In [ ]:
%%writefile test.py
n = int(input())
A = [input().split() for _ in range(n)] # n行読み込む。
print(A)

for i in range(n): A[i][1] = int(A[i][1]) # 右の要素を整数にする。
print(A)

stableSortedA = sorted(A, key=lambda x: x[1]) # 右の要素でソートする。
for a in stableSortedA: print(*a)

In [ ]:
!python3 test.py < input.dat

### 本題


♠クイックソートは分割方法によっていくつかのバリエーションがある。この問題は，教科書と同じ分割方法を使わないとACにならない。例えば，NumPyの`np.sort`は`kind='quicksort'`とするとクイックソートになるが，分割方法が教科書とは異なるため，ここでは使えない。

教科書の擬似コードをPythonに翻訳すると，次のようなクイックソートになる。

In [ ]:
def partition(A, p, r):
  x = A[r]
  i = p - 1
  for j in range(p, r):
    if A[j] <= x:
      i += 1
      A[i], A[j] = A[j], A[i]
  A[i + 1], A[r] = A[r], A[i + 1]
  return i + 1

def quickSort(A, p, r):
  if p < r:
    q = partition(A, p, r)
    quickSort(A, p, q - 1)
    quickSort(A, q + 1, r)

A = [13, 19, 9, 5, 12, 8, 7, 4, 21, 2, 6, 11]
quickSort(A, 0, len(A) - 1)
A

この関数で整列できるのは数値のリストであって，この問題での入力データは，そのままでは整列できない。次のような対策がある。

-   （対策1）`partition`の中で，比較に使うデータを`A[r][1]`のように変更する。（長所：単純。短所：数のリストのソートができなくなる。）
-   ♠（対策2）絵柄と数をまとめるクラス`Element`を作り，次のメソッドを定義する。（長所：既存の関数には変更がない。短所：面倒）
    -   `partition`で使う`<=`のためのメソッド`__le__`
    -   安定かどうかの確認に使う`==`のためのメソッド`__eq__`
    -   print用のメソッド`__str__`

第1の対策は単純だから自分でやってみる。

♠第2の対策のためのコードは次のようになる。

In [ ]:
# partitionは変更無し

# quickSortは変更無し

class Element:
  def __init__(self, pair):
    self.suit = pair[0]  # 絵柄
    self.number = pair[1]  # 数

  def __le__(self, other):
    return self.number <= other.number  # <= のためのメソッド

  def __eq__(self, other): # == のためのメソッド
    return self.suit == other.suit and self.number == other.number

  def __str__(self): # print用のメソッド
    return f"{self.suit} {self.number}"

n = int(input())
A = [input().split() for _ in range(n)]
for i in range(n): A[i][1] = int(A[i][1])
A = [Element(a) for a in A] # Elementのリストに変換する。

stableSortedA = sorted(A, key=lambda x: x.number) # 安定な整列（keyに注意）

quickSort(A, 0, len(A) - 1) # クイックソート（破壊的だから後）

if A == stableSortedA: print('Stable')
else: print('Not stable')
for a in A: print(a)

## 7.4 計数ソート


問題：[ALDS1_6_A](https://onlinejudge.u-aizu.ac.jp/courses/lesson/1/ALDS1/6/ALDS1_6_A) (Counting Sort)

クイックソートではTLEになる。（Pythonの`.sort`や`sorted`を使えばACになるが，そんなことをしてもしょうがない。）

In [ ]:
import sys
sys.setrecursionlimit(10**6) # 再帰の関数制限の緩和

def partition(A, p, r):
  x = A[r]
  i = p - 1
  for j in range(p, r):
    if A[j] <= x:
      i += 1
      A[i], A[j] = A[j], A[i]
  A[i + 1], A[r] = A[r], A[i + 1]
  return i + 1

def quickSort(A, p, r):
  if p < r:
    q = partition(A, p, r)
    quickSort(A, p, q - 1)
    quickSort(A, q + 1, r)

n = int(input())
A = list(map(int, input().split()))
quickSort(A, 0, len(A) - 1)
print(*A)

計数ソートを実装する。0オリジンで実装する場合，教科書の`B[C[A[j]]] = A[j]`は`B[C[A[j]] - 1] = A[j]`になる。

In [ ]:
def CountingSort(A):
  n = len(A)
  k = 10000
  B = [0] * n
  C = [0] * (k + 1)

  #後は自分で書いてみる。

## ♠7.6 反転数


問題：[ALDS1_5_D](https://onlinejudge.u-aizu.ac.jp/courses/lesson/1/ALDS1/5/ALDS1_5_D) (The Number of Inversions)

## ♠7.7 最小コストソート


問題：[ALDS1_6_D](https://onlinejudge.u-aizu.ac.jp/courses/lesson/1/ALDS1/6/ALDS1_6_D) (Minimum Cost Sort)

## 宿題


以下の問題をAC（Accepted）にする。Pythonを使うこと。

-   [ALDS1_5_B](https://onlinejudge.u-aizu.ac.jp/courses/lesson/1/ALDS1/5/ALDS1_5_B) (Merge Sort)
-   [ALDS1_6_B](https://onlinejudge.u-aizu.ac.jp/courses/lesson/1/ALDS1/6/ALDS1_6_B) (Partition)
-   [ALDS1_6_C](https://onlinejudge.u-aizu.ac.jp/courses/lesson/1/ALDS1/6/ALDS1_6_C) (Quick Sort)
-   [ALDS1_6_A](https://onlinejudge.u-aizu.ac.jp/courses/lesson/1/ALDS1/6/ALDS1_6_A) (Counting Sort)
-   ♠[ALDS1_5_D](https://onlinejudge.u-aizu.ac.jp/courses/lesson/1/ALDS1/5/ALDS1_5_D) (The Number of Inversions)
-   ♠[ALDS1_6_D](https://onlinejudge.u-aizu.ac.jp/courses/lesson/1/ALDS1/6/ALDS1_6_D) (Minimum Cost Sort)

以上